# Analysis

In [6]:
import json
import re
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

output_dir = 'results/target_docs/old/compare'
output_file = f'{output_dir}/direct_query-nfcorpus-llama3-Top15-M250-N15.json'
with open(output_file, 'r') as f:
    data = json.load(f)

def extract_yes_no(answer):
    """Extract 'Yes' or 'No' from a response."""
    match = re.search(r'\b(Yes|No)\b', answer, re.IGNORECASE)
    return match.group(1).capitalize() if match else "Invalid"

def load_and_calculate_metrics(data):
    expected_answers = []
    predicted_answers = []

    for doc_id, doc_data in data.items():
        expected_answer = "Yes" if doc_data["mem"].lower() == "yes" else "No"
        llm_response = extract_yes_no(doc_data["llm_responses"][0])

        # Skip invalid responses
        if llm_response == "Invalid":
            continue

        expected_answers.append(expected_answer)
        predicted_answers.append(llm_response)

    # Calculate metrics
    accuracy = accuracy_score(expected_answers, predicted_answers) * 100
    precision = precision_score(expected_answers, predicted_answers, pos_label="Yes", zero_division=0)
    recall = recall_score(expected_answers, predicted_answers, pos_label="Yes", zero_division=0)
    f1 = f1_score(expected_answers, predicted_answers, pos_label="Yes", zero_division=0)

    return accuracy, precision, recall, f1

# Calculate and print metrics
accuracy, precision, recall, f1 = load_and_calculate_metrics(data)
print(f"Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Accuracy: 75.00%
Precision: 0.67
Recall: 0.98
F1 Score: 0.80


# Compare

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import json
import os
import re

# Dataset and results directory
output_dir = 'results/target_docs/old/compare'

# Check if file exists
def file_exists(filepath):
    return os.path.isfile(filepath)

# Function to extract 'Yes' or 'No' from Anderson et al.'s responses
def extract_yes_no(answer):
    match = re.search(r'\b(Yes|No)\b', answer, re.IGNORECASE)
    return match.group(1).capitalize() if match else "Invalid"

# Function to calculate a single ROC point for Anderson et al.
def compute_anderson_point(data):
    """Compute metrics and return a single ROC point."""
    expected_answers, predicted_answers = [], []

    for doc_id, doc_data in data.items():
        expected_answer = 1 if doc_data["mem"].lower() == "yes" else 0
        llm_response = extract_yes_no(doc_data["llm_responses"][0])
        if llm_response == "Invalid":
            continue

        predicted_answer = 1 if llm_response == "Yes" else 0
        expected_answers.append(expected_answer)
        predicted_answers.append(predicted_answer)

    # Calculate metrics
    accuracy = accuracy_score(expected_answers, predicted_answers) * 100
    precision = precision_score(expected_answers, predicted_answers, zero_division=0)
    recall = recall_score(expected_answers, predicted_answers, zero_division=0)
    f1 = f1_score(expected_answers, predicted_answers, zero_division=0)

    # ROC point
    fpr = 1 - precision  # False Positive Rate
    tpr = recall  # True Positive Rate

    return accuracy, precision, recall, f1, fpr, tpr

# Mock function for other attacks' scores
def get_mia_scores(data):
    all_acc = []
    for doc_id, doc_data in data.items():
        correct_answers = doc_data['answers']
        llm_responses = doc_data['llm_responses']
        total_questions = len(correct_answers)
        correct_count = 0

        # Compare LLM responses with correct answers
        for i in range(total_questions):
            correct_answer = extract_yes_no(correct_answers[i])
            llm_answer = extract_yes_no(llm_responses[i])
            if correct_answer == llm_answer:
                correct_count += 1

        # Calculate accuracy for the document
        accuracy = correct_count / total_questions * 100
        all_acc.append(accuracy)
    return all_acc

# Function to plot ROC curves (with or without log scale)
def plot_roc_curves(dataset, attacks_data, anderson_point=None, log_scale=False):
    """Plot TPR vs. FPR with or without log-log axes."""
    plt.figure(figsize=(4, 3), dpi=200)

    for fpr, tpr, label in attacks_data:
        plt.plot(fpr, tpr, label=label, alpha=0.7)

    # Add Anderson et al.'s point if available
    if anderson_point:
        acc, prec, rec, f1, fpr, tpr = anderson_point
        plt.scatter([fpr], [tpr], color='red', label=f"Anderson et al.", zorder=5)

    # Set log-log scale if required
    if log_scale:
        plt.semilogx()
        plt.semilogy()
        plt.xlim(1e-3, 1)
        plt.ylim(1e-3, 1)

    # Add axis labels and legend
    plt.xlabel("False Positive Rate", fontsize=10)
    plt.ylabel("True Positive Rate", fontsize=10)
    plt.plot([0, 1], [0, 1], ls='--', color='gray', alpha=0.5)
    plt.legend(loc="best", fontsize=8)
    plt.title(dataset.upper(), fontsize=12)
    plt.savefig(f'{dataset}_{"log" if log_scale else "regular"}.png', bbox_inches='tight')
    plt.show()

# Prepare data for plotting
datasets = ['nfcorpus', 'trec-covid']
roc_data_by_dataset = {dataset: [] for dataset in datasets}
anderson_points = {}

for dataset in datasets:
    # Process Attack 1: Your attack
    for top in [15]:
        output_file = f'{output_dir}/mia-{dataset}-llama3-Top{top}-M250-N15.json'
        if file_exists(output_file):
            with open(output_file, 'r') as f:
                data = json.load(f)

            all_acc = get_mia_scores(data)
            mem_acc = all_acc[:len(all_acc)//2]
            nonmem_acc = all_acc[len(all_acc)//2:]

            labels = np.concatenate([np.ones(len(mem_acc)), np.zeros(len(nonmem_acc))])
            scores = np.concatenate([mem_acc, nonmem_acc])

            fpr, tpr, _ = roc_curve(labels, scores)
            roc_auc = auc(fpr, tpr)
            roc_data_by_dataset[dataset].append((fpr, tpr, f"Ours (AUC={roc_auc:.3f})"))
        else:
            print(f"File not found: {output_file}")

    # Process Attack 2: Li et al.
    output_file = f'{output_dir}/s2-{dataset}-llama3-Top15-M250-N15.json'
    if file_exists(output_file):
        with open(output_file, 'r') as f:
            data = json.load(f)

        all_acc = [doc_data['bleu_score'] for doc_data in data.values()]
        mem_acc = all_acc[:len(all_acc)//2]
        nonmem_acc = all_acc[len(all_acc)//2:]

        labels = np.concatenate([np.ones(len(mem_acc)), np.zeros(len(nonmem_acc))])
        scores = np.concatenate([mem_acc, nonmem_acc])

        fpr, tpr, _ = roc_curve(labels, scores)
        roc_auc = auc(fpr, tpr)
        roc_data_by_dataset[dataset].append((fpr, tpr, f"Li et al. (AUC={roc_auc:.3f})"))
    else:
        print(f"File not found: {output_file}")

    # Process Attack 3: Anderson et al.
    output_file = f'{output_dir}/direct_query-{dataset}-llama3-Top15-M250-N15.json'
    if file_exists(output_file):
        with open(output_file, 'r') as f:
            data = json.load(f)

        anderson_points[dataset] = compute_anderson_point(data)
    else:
        print(f"File not found: {output_file}")

# Plot for each dataset in both regular and log-log format
for dataset, roc_data in roc_data_by_dataset.items():
    formatted_data = [(fpr, tpr, label) for fpr, tpr, label in roc_data]

    # Regular ROC plot
    plot_roc_curves(dataset, formatted_data, anderson_points.get(dataset), log_scale=False)

    # Log-log ROC plot
    plot_roc_curves(dataset, formatted_data, anderson_points.get(dataset), log_scale=True)



Anderson et al. Metrics:
Accuracy: 75.00%
Precision: 0.67
Recall: 0.98
F1 Score: 0.80
False Positive Rate (FPR): 0.330
True Positive Rate (TPR): 0.983

Dataset: NFCORPUS
Anderson et al. - FPR: 0.330, TPR: 0.983
Ours (Top-15) AUC=0.965 - TPR at Anderson's FPR: 0.995
Li et al. AUC=0.749 - TPR at Anderson's FPR: 0.696


In [12]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import json
import os
import re

# Dataset and results directory
output_dir = 'results/target_docs/old/compare'

# Check if file exists
def file_exists(filepath):
    return os.path.isfile(filepath)

# Function to extract 'Yes' or 'No' from Anderson et al.'s responses
def extract_yes_no(answer):
    match = re.search(r'\b(Yes|No)\b', answer, re.IGNORECASE)
    return match.group(1).capitalize() if match else "Invalid"

# Function to calculate a single ROC point for Anderson et al.
def compute_anderson_point(data):
    """Compute metrics and return a single ROC point."""
    expected_answers, predicted_answers = [], []

    for doc_id, doc_data in data.items():
        expected_answer = 1 if doc_data["mem"].lower() == "yes" else 0
        llm_response = extract_yes_no(doc_data["llm_responses"][0])
        if llm_response == "Invalid":
            continue

        predicted_answer = 1 if llm_response == "Yes" else 0
        expected_answers.append(expected_answer)
        predicted_answers.append(predicted_answer)

    # Calculate metrics
    accuracy = accuracy_score(expected_answers, predicted_answers) * 100
    precision = precision_score(expected_answers, predicted_answers, zero_division=0)
    recall = recall_score(expected_answers, predicted_answers, zero_division=0)
    f1 = f1_score(expected_answers, predicted_answers, zero_division=0)

    # ROC point
    fpr = 1 - precision  # False Positive Rate
    tpr = recall  # True Positive Rate

    # Print Anderson et al.'s metrics
    print("\nAnderson et al. Metrics:")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"False Positive Rate (FPR): {fpr:.3f}")
    print(f"True Positive Rate (TPR): {tpr:.3f}")

    return accuracy, precision, recall, f1, fpr, tpr

# Interpolate TPR at specific FPR for other methods
def interpolate_tpr(fpr_target, fpr_list, tpr_list):
    return np.interp(fpr_target, fpr_list, tpr_list)

# Mock function for other attacks' scores
def get_mia_scores(data):
    all_acc = []
    for doc_id, doc_data in data.items():
        correct_answers = doc_data['answers']
        llm_responses = doc_data['llm_responses']
        total_questions = len(correct_answers)
        correct_count = 0

        # Compare LLM responses with correct answers
        for i in range(total_questions):
            correct_answer = extract_yes_no(correct_answers[i])
            llm_answer = extract_yes_no(llm_responses[i])
            if correct_answer == llm_answer:
                correct_count += 1

        # Calculate accuracy for the document
        accuracy = correct_count / total_questions * 100
        all_acc.append(accuracy)
    return all_acc

# Prepare data for plotting and metrics comparison
datasets = ['nfcorpus']
roc_data_by_dataset = {dataset: [] for dataset in datasets}
anderson_points = {}

low_fprs = [0.001, 0.01]  # Low FPR thresholds: 0.1% and 1%

for dataset in datasets:
    # Process Attack 1: Your attack
    for top in [15]:
        output_file = f'{output_dir}/mia-{dataset}-llama3-Top{top}-M250-N15.json'
        if file_exists(output_file):
            with open(output_file, 'r') as f:
                data = json.load(f)

            all_acc = get_mia_scores(data)
            mem_acc = all_acc[:len(all_acc)//2]
            nonmem_acc = all_acc[len(all_acc)//2:]

            labels = np.concatenate([np.ones(len(mem_acc)), np.zeros(len(nonmem_acc))])
            scores = np.concatenate([mem_acc, nonmem_acc])

            fpr, tpr, _ = roc_curve(labels, scores)
            roc_auc = auc(fpr, tpr)
            roc_data_by_dataset[dataset].append((fpr, tpr, f"Ours (Top-{top}) AUC={roc_auc:.3f}"))

            # Report TPR at low FPRs
            for low_fpr in low_fprs:
                tpr_at_low_fpr = interpolate_tpr(low_fpr, fpr, tpr)
                print(f"Ours (Top-{top}) - TPR at FPR={low_fpr:.3%}: {tpr_at_low_fpr:.3f}")
        else:
            print(f"File not found: {output_file}")

    # Process Attack 2: Li et al.
    output_file = f'{output_dir}/s2-{dataset}-llama3-Top15-M250-N15.json'
    if file_exists(output_file):
        with open(output_file, 'r') as f:
            data = json.load(f)

        all_acc = [doc_data['bleu_score'] for doc_data in data.values()]
        mem_acc = all_acc[:len(all_acc)//2]
        nonmem_acc = all_acc[len(all_acc)//2:]

        labels = np.concatenate([np.ones(len(mem_acc)), np.zeros(len(nonmem_acc))])
        scores = np.concatenate([mem_acc, nonmem_acc])

        fpr, tpr, _ = roc_curve(labels, scores)
        roc_auc = auc(fpr, tpr)
        roc_data_by_dataset[dataset].append((fpr, tpr, f"Li et al. AUC={roc_auc:.3f}"))

        # Report TPR at low FPRs
        for low_fpr in low_fprs:
            tpr_at_low_fpr = interpolate_tpr(low_fpr, fpr, tpr)
            print(f"Li et al. - TPR at FPR={low_fpr:.3%}: {tpr_at_low_fpr:.3f}")
    else:
        print(f"File not found: {output_file}")

    # Process Attack 3: Anderson et al.
    output_file = f'{output_dir}/direct_query-{dataset}-llama3-Top15-M250-N15.json'
    if file_exists(output_file):
        with open(output_file, 'r') as f:
            data = json.load(f)

        anderson_points[dataset] = compute_anderson_point(data)
    else:
        print(f"File not found: {output_file}")

# Report Anderson et al.'s FPR and TPR, and interpolate TPR for other methods
for dataset, roc_data in roc_data_by_dataset.items():
    print(f"\nDataset: {dataset.upper()}")

    # Report Anderson et al.'s metrics
    if dataset in anderson_points:
        _, _, _, _, anderson_fpr, anderson_tpr = anderson_points[dataset]
        print(f"Anderson et al. - FPR: {anderson_fpr:.3f}, TPR: {anderson_tpr:.3f}")

        # Interpolate TPR for other methods at Anderson's FPR
        for fpr, tpr, label in roc_data:
            interpolated_tpr = interpolate_tpr(anderson_fpr, fpr, tpr)
            print(f"{label} - TPR at Anderson's FPR: {interpolated_tpr:.3f}")

        # Report TPR at low FPRs
        for low_fpr in low_fprs:
            for fpr, tpr, label in roc_data:
                tpr_at_low_fpr = interpolate_tpr(low_fpr, fpr, tpr)
                print(f"{label} - TPR at FPR={low_fpr:.3%}: {tpr_at_low_fpr:.3f}")


Ours (Top-15) - TPR at FPR=0.100%: 0.293
Ours (Top-15) - TPR at FPR=1.000%: 0.406
Li et al. - TPR at FPR=0.100%: 0.012
Li et al. - TPR at FPR=1.000%: 0.072

Anderson et al. Metrics:
Accuracy: 75.00%
Precision: 0.67
Recall: 0.98
F1 Score: 0.80
False Positive Rate (FPR): 0.330
True Positive Rate (TPR): 0.983

Dataset: NFCORPUS
Anderson et al. - FPR: 0.330, TPR: 0.983
Ours (Top-15) AUC=0.965 - TPR at Anderson's FPR: 0.995
Li et al. AUC=0.749 - TPR at Anderson's FPR: 0.696
Ours (Top-15) AUC=0.965 - TPR at FPR=0.100%: 0.293
Li et al. AUC=0.749 - TPR at FPR=0.100%: 0.012
Ours (Top-15) AUC=0.965 - TPR at FPR=1.000%: 0.406
Li et al. AUC=0.749 - TPR at FPR=1.000%: 0.072
